# About
This script is used to convert DeepLabV3_MNV2_Pascal to Core ML

# Download model and setup enviroments

In [1]:
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
!tar -zxvf deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz

--2021-08-10 05:37:25--  http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.203.128, 2404:6800:4008:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.203.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23916119 (23M) [application/x-tar]
Saving to: ‘deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz’

deeplabv3_mnv2_pasc 100%[===================>]  22.81M  27.3MB/s    in 0.8s    

2021-08-10 05:37:28 (27.3 MB/s) - ‘deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz’ saved [23916119/23916119]

deeplabv3_mnv2_pascal_trainval/
deeplabv3_mnv2_pascal_trainval/model.ckpt-30000.index
deeplabv3_mnv2_pascal_trainval/frozen_inference_graph.pb
deeplabv3_mnv2_pascal_trainval/model.ckpt-30000.data-00000-of-00001


In [2]:
!pip install coremltools==4.1
!pip install tfcoreml==2.0.0

     |████████████████████████████████| 3.4 MB 4.3 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=c1273c855e3518d75530db4cb55316db87b3fb8517ac2b5902b99c94149c2e1c
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr
     |████████████████████████████████| 44 kB 1.2 MB/s 
     |████████████████████████████████| 109.3 MB 14 kB/s 
     |████████████████████████████████| 3.1 MB 51.3 MB/s 
     |████████████████████████████████| 488 kB 49.9 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
 

In [3]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


4.1


# Convert from TF to Core ML

In [4]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    return graph_def

def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))
  print('Exported to:', filename)


model_name = 'deeplabv3_mnv2_pascal_trainval'
input_file = f'{model_name}/frozen_inference_graph.pb'
with tf.Session(graph=tf.Graph()) as sess:
  graph_def = load_frozen_graph(input_file)
  tf.import_graph_def(graph_def, name="")
  the_graph = tf.compat.v1.get_default_graph()
  export_ops_name(the_graph, f'{model_name}_ops.txt')



Exported to: deeplabv3_mnv2_pascal_trainval_ops.txt


In [5]:
# https://github.com/hollance/coreml-survival-guide/blob/master/Scripts/deeplab.py

import tfcoreml
import coremltools as ct


def get_nn(spec):
    if spec.WhichOneof("Type") == "neuralNetwork":
        return spec.neuralNetwork
    elif spec.WhichOneof("Type") == "neuralNetworkClassifier":
        return spec.neuralNetworkClassifier
    elif spec.WhichOneof("Type") == "neuralNetworkRegressor":
        return spec.neuralNetworkRegressor
    else:
        raise ValueError("MLModel does not have a neural network")


def find_layer_index(name):
    for i, layer in enumerate(nn.layers):
        if layer.name == name:
            return i
    return None


# Convert the TensorFlow model to Core ML.

input_path = "deeplabv3_mnv2_pascal_trainval/frozen_inference_graph.pb"
output_path = "DeepLabV3_1.mlmodel"

input_tensor = "ImageTensor:0"
input_name = "ImageTensor__0"
output_tensor = "ResizeBilinear_3:0"

model = tfcoreml.convert(tf_model_path=input_path,
                         mlmodel_path=output_path,
                         output_feature_names=[output_tensor],
                         input_name_shape_dict={input_tensor: [1, 513, 513, 3]},
                         image_input_names=input_name)

print(model)


Loading the TF graph...
Graph Loaded.
Now finding ops in the TF graph that can be dropped for inference
Done.
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob ImageTensor:0
1/970: Analysing op name: SemanticPredictions/size/0 ( type:  Const )
2/970: Analysing op name: SemanticPredictions/begin ( type:  Const )
3/970: Analysing op name: strided_slice_9/stack_2 ( type:  Const )
4/970: Analysing op name: strided_slice_9/stack_1 ( type:  Const )
5/970: Analysing op name: strided_slice_9/stack ( type:  Const )
6/970: Analysing op name: strided_slice_8/stack_2 ( type:  Const )
7/970: Analysing op name: strided_slice_8/stack_1 ( type:  Const )
8/970: Analysing op name: strided_slice_8/stack ( type:  Const )
9/970: Analysing op name: ArgMax/dimension ( type:  Const )
10/970: Analysing op name: strided_slice_7/stack_2 ( type:  Const )
11/970: Analysing op name: strided_slice_7/stack_1 ( type:  Const )
12/970: Analysing op name: strided_slice_7/s

In [6]:
# Fill in the descriptions and metadata.

spec = model._spec

spec.description.metadata.versionString = "v1.0"
spec.description.metadata.shortDescription = "DeepLab v3+ on MobileNet v2"
spec.description.metadata.author = "https://github.com/tensorflow/models/tree/master/research/deeplab"
spec.description.metadata.license = "Apache License"

# Rename inputs and outputs.

old_input_name = "ImageTensor__0"
new_input_name = "image"
old_output_name = "ResizeBilinear_3__0"
new_output_name = "scores"

spec.description.input[0].name = new_input_name
spec.description.input[0].shortDescription = "Input image"
spec.description.output[0].name = new_output_name
spec.description.output[0].shortDescription = "Segmentation map"

nn = get_nn(spec)

for i in range(len(nn.layers)):
    if len(nn.layers[i].input) > 0: 
        if nn.layers[i].input[0] == old_input_name:
            nn.layers[i].input[0] = new_input_name
    if len(nn.layers[i].output) > 0: 
        if nn.layers[i].output[0] == old_output_name:
            nn.layers[i].output[0] = new_output_name

spec.neuralNetwork.preprocessing[0].featureName = new_input_name

# Remove the second-to-last layer.
print(nn.layers[-1])
print(nn.layers[-2])

nn.layers[-1].input[0] = nn.layers[-2].input[0]
del nn.layers[-2]

# Remove the unused layers at the beginning.

resize_layer = nn.layers[find_layer_index("ResizeBilinear:0")]
multiply_layer = nn.layers[find_layer_index("mul_1:0")]
multiply_layer.input[0] = resize_layer.input[0]

del nn.layers[find_layer_index("ResizeBilinear:0")]
del nn.layers[find_layer_index("negated_Reshape:0_sub_2:0")]
del nn.layers[find_layer_index("sub_2:0")]
del nn.layers[find_layer_index("Pad:0")]
del nn.layers[find_layer_index("Reshape/tensor:0")]
del nn.layers[find_layer_index("add_2:0")]

# Replace preprocessing layers.

nn.preprocessing[0].scaler.channelScale = 1/127.5
nn.preprocessing[0].scaler.redBias = -1.0
nn.preprocessing[0].scaler.greenBias = -1.0
nn.preprocessing[0].scaler.blueBias = -1.0

conv_layer = nn.layers[find_layer_index("MobilenetV2/Conv/Conv2D:0")]
conv_layer.input[0] = multiply_layer.input[0]

del nn.layers[find_layer_index("mul_1:0")]
del nn.layers[find_layer_index("sub_7:0")]

# Add an argmax layer at the end.

new_layer = nn.layers.add()
new_layer.name = "argmax"
params = ct.proto.NeuralNetwork_pb2.ReduceLayerParams
new_layer.reduce.mode = params.ARGMAX
new_layer.reduce.axis = params.C

new_layer.output.append(nn.layers[-2].output[0])
nn.layers[-2].output[0] = nn.layers[-2].name + "_output"
new_layer.input.append(nn.layers[-2].output[0])

# Fix up the output shape and make it INT32.

del spec.description.output[0].type.multiArrayType.shape[0]
spec.description.output[0].type.multiArrayType.dataType = ct.proto.FeatureTypes_pb2.ArrayFeatureType.INT32

# Convert weights to 16 bit floats and save the model.

spec = ct.utils.convert_neural_network_spec_weights_to_fp16(spec)
ct.models.utils.save_spec(spec, "DeepLabV3_Clean.mlmodel")

name: "ResizeBilinear_3:0"
input: "ResizeBilinear_2:0"
output: "scores"
resizeBilinear {
  targetSize: 513
  targetSize: 513
  mode {
  }
}

name: "ResizeBilinear_2:0"
input: "logits/semantic/BiasAdd:0"
output: "ResizeBilinear_2:0"
resizeBilinear {
  targetSize: 65
  targetSize: 65
  mode {
  }
}

Quantizing layer MobilenetV2/Conv/Conv2D:0
Quantizing layer MobilenetV2/Conv/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv/depthwise/depthwise:0
Quantizing layer MobilenetV2/expanded_conv/depthwise/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv/project/Conv2D:0
Quantizing layer MobilenetV2/expanded_conv/project/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_1/expand/Conv2D:0
Quantizing layer MobilenetV2/expanded_conv_1/expand/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_1/depthwise/depthwise:0
Quantizing layer MobilenetV2/expanded_conv_1/depthwise/BatchNorm/FusedBatchNorm:0
Quantizing layer Mobilene

/usr/local/lib/python3.7/dist-packages/coremltools/models/_deprecation.py:25: FutureWarning: Function "convert_neural_network_spec_weights_to_fp16" is deprecated and will be removed in the next release; instead use 'coremltools.models.neural_network.quantization_utils.quantize_weights'.
  warnings.warn(msg, category=FutureWarning)


Quantizing layer MobilenetV2/expanded_conv_9/expand/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_9/depthwise/depthwise:0
Quantizing layer MobilenetV2/expanded_conv_9/depthwise/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_9/project/Conv2D:0
Quantizing layer MobilenetV2/expanded_conv_9/project/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_10/expand/Conv2D:0
Quantizing layer MobilenetV2/expanded_conv_10/expand/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_10/depthwise/depthwise:0
Quantizing layer MobilenetV2/expanded_conv_10/depthwise/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_10/project/Conv2D:0
Quantizing layer MobilenetV2/expanded_conv_10/project/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_11/expand/Conv2D:0
Quantizing layer MobilenetV2/expanded_conv_11/expand/BatchNorm/FusedBatchNorm:0
Quantizing layer MobilenetV2/expanded_conv_11/depth

In [7]:
print(spec.description)

input {
  name: "image"
  shortDescription: "Input image"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Segmentation map"
  type {
    multiArrayType {
      shape: 513
      shape: 513
      dataType: INT32
    }
  }
}
metadata {
  shortDescription: "DeepLab v3+ on MobileNet v2"
  versionString: "v1.0"
  author: "https://github.com/tensorflow/models/tree/master/research/deeplab"
  license: "Apache License"
}



In [8]:
import json

# load the model
mlmodel = ct.models.MLModel(spec)

labels_json = {"labels": ["background", "aeroplane", "bicycle", "bird", "board", "bottle", "bus", "car", "cat", "chair", "cow", "diningTable", "dog", "horse", "motorbike", "person", "pottedPlant", "sheep", "sofa", "train", "tvOrMonitor"]}

mlmodel.user_defined_metadata["com.apple.coreml.model.preview.type"] = "imageSegmenter"
mlmodel.user_defined_metadata['com.apple.coreml.model.preview.params'] = json.dumps(labels_json)

mlmodel.save("DeepLabV3_Pascal.mlmodel")
print(mlmodel)

input {
  name: "image"
  shortDescription: "Input image"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Segmentation map"
  type {
    multiArrayType {
      shape: 513
      shape: 513
      dataType: INT32
    }
  }
}
metadata {
  shortDescription: "DeepLab v3+ on MobileNet v2"
  versionString: "v1.0"
  author: "https://github.com/tensorflow/models/tree/master/research/deeplab"
  license: "Apache License"
  userDefined {
    key: "com.apple.coreml.model.preview.params"
    value: "{\"labels\": [\"background\", \"aeroplane\", \"bicycle\", \"bird\", \"board\", \"bottle\", \"bus\", \"car\", \"cat\", \"chair\", \"cow\", \"diningTable\", \"dog\", \"horse\", \"motorbike\", \"person\", \"pottedPlant\", \"sheep\", \"sofa\", \"train\", \"tvOrMonitor\"]}"
  }
  userDefined {
    key: "com.apple.coreml.model.preview.type"
    value: "imageSegmenter"
  }
}

